# Feature_generation

### Loading required libraries 

In [1]:
# matrices
import numpy as np
import pandas as pd

#sklearn
from sklearn.model_selection import train_test_split


#### Loading prepared dataset

In [2]:
X = pd.read_csv('../../data/intermid/X_wo_missed_values.csv',index_col=0)
y = pd.read_csv('../../data/intermid/y_wo_missed_values.csv',index_col=0)

### Feature generation. 


In [3]:
df = pd.concat([X,y],axis=1)

### Making samples by MEAN OF TWO RANDOM CHOSEN SAMPLES

In [4]:
index_high_quality = df[df.quality==2].index

In [5]:
def make_new_sample(n,dataset):
    '''
        MAKE NEW SAMPLE FROM MEAN OF TWO RANDOM CHOSEN SAMPLES
        N - new samples amount
    '''
    for i in range(n):
        index_1, index_2 = np.random.randint(low=0,high=len(dataset)-1,size=2)
        b = np.array(dataset)
        new_item = (b[index_1]+b[index_2])/2
        new_item = pd.DataFrame(new_item.reshape(1,-1), columns=dataset.columns)
        dataset = dataset.append(new_item)
    dataset.reset_index(inplace=True)
    dataset.drop(columns='index',inplace=True)
    return dataset


In [6]:
high_quality_df = df[df.quality==2]

In [7]:
high_quality_df_pH = list(high_quality_df.pH)

In [8]:
a = make_new_sample(3000,high_quality_df)

In [9]:
a['check'] = a.pH.apply(lambda x: 1 if x in high_quality_df_pH else 0)
np.bincount(a.check)

array([2994,   11], dtype=int64)

In [10]:
new_high_quality_samples = a.loc[a.check == 0,:].iloc[:,:-1]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(index=index_high_quality).iloc[:,:-1],
                                                    df.drop(index=index_high_quality).iloc[:,-1],test_size=.4,random_state=255)

#adding high quality wines
X_test = X_test.append(df.iloc[index_high_quality,:-1])
y_test = y_test.append(df.iloc[index_high_quality,-1])

X_test, X_val, y_test, y_val = train_test_split(X_test,y_test,test_size=.4,random_state=255)

np.bincount(y_train), np.bincount(y_test),np.bincount(y_val)

(array([ 137, 3758], dtype=int64),
 array([  64, 1494,    3], dtype=int64),
 array([ 45, 994,   2], dtype=int64))

In [12]:
#adding generated examples to train dataset 
X_train = X_train.append(new_high_quality_samples.iloc[:,:-1])
y_train = y_train.append(new_high_quality_samples.iloc[:,-1])

np.bincount(y_train), np.bincount(y_test),np.bincount(y_val)

(array([ 137, 3758, 2994], dtype=int64),
 array([  64, 1494,    3], dtype=int64),
 array([ 45, 994,   2], dtype=int64))

In [13]:
#making 100 duplicates of high_quality wine to X_test and y_test:
X_y_test = pd.concat([X_test,y_test],axis=1)

a = X_y_test[X_y_test.quality==2]

a = a.sample(n=47,random_state=255,replace=True)

X_y_test = pd.concat([a,X_y_test],axis=0)


X_test = X_y_test.iloc[:,:-1]
y_test = X_y_test.iloc[:,-1]

np.bincount(y_test)

array([  64, 1494,   50], dtype=int64)

In [14]:
#making 100 duplicates of high_quality wine to X_val and y_val:
X_y_val = pd.concat([X_val,y_val],axis=1)

a = X_y_val[X_y_val.quality==2]

a = a.sample(n=48,random_state=255,replace=True)

X_y_val = pd.concat([a,X_y_val],axis=0)


X_val = X_y_val.iloc[:,:-1]
y_val = X_y_val.iloc[:,-1]

np.bincount(y_val)

array([ 45, 994,  50], dtype=int64)

In [15]:
print('class:')
print('0\t1\t2')
print('-'*35)
print(np.bincount(y_train), ' <- train')
print(np.bincount(y_test), ' <- test')
print(np.bincount(y_val), ' <- validation')

class:
0	1	2
-----------------------------------
[ 137 3758 2994]  <- train
[  64 1494   50]  <- test
[ 45 994  50]  <- validation


In [16]:
np.bincount(y_train), np.bincount(y_test),np.bincount(y_val)

(array([ 137, 3758, 2994], dtype=int64),
 array([  64, 1494,   50], dtype=int64),
 array([ 45, 994,  50], dtype=int64))

In [17]:
#saving to file:
X_train.to_csv('X_train.csv')
y_train.to_csv('y_train.csv')

X_test.to_csv('X_test.csv')
y_test.to_csv('y_test.csv')

X_val.to_csv('X_val.csv')
y_val.to_csv('y_val.csv')